In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-01 11:59:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.154.99.47, 18.154.99.177, 18.154.99.220, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.154.99.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  45.5MB/s    in 1.3s    

2025-03-01 11:59:43 (45.5 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-01 13:04:23--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.154.99.220, 18.154.99.177, 18.154.99.225, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.154.99.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-01 13:04:23 (226 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [ ]:
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ny_taxi_parquet").getOrCreate()

# loading trip data
df = spark.read.parquet('./yellow_tripdata_2024-10.parquet')

In [ ]:
# loading taxi zone data
zone_df = spark.read.csv('./taxi_zone_lookup.csv', header='true')

In [ ]:
# ensure correct format for each column
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [ ]:
zone_df.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [ ]:
# Q1: Repartition the Dataframe to 4 partitions and save it to parquet.
partition_df = df.repartition(4)

# write into parquet file
df.write.parquet("YellowTaxi_partition")

#### **Question 1.**: What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

```Ans: 25MB```

In [ ]:
#
df.filter(
      df.tpep_pickup_datetime.between('2024-10-15 00:00:00', '2024-10-16 00:00:00') &
      df.tpep_dropoff_datetime.between('2024-10-15 00:00:00', '2024-10-16 00:00:00'))\
  .select(f.count(f.lit(1)).alias("Num Of Records")).show()

+--------------+
|Num Of Records|
+--------------+
|        128013|
+--------------+



#### **Question 3**: How many taxi trips were there on the 15th of October?

```Ans: ```

In [ ]:
# calculate each trip duration (hours)
df = df.withColumn(
    "trip_duration_hours",
    ((f.col("tpep_dropoff_datetime") - f.col("tpep_pickup_datetime")).cast("long")) / 3600
)

# getting the longest trips (hours)
df.agg(
    f.max("trip_duration_hours").alias("longest_trip_in_hours")
).show()

+---------------------+
|longest_trip_in_hours|
+---------------------+
|   162.61777777777777|
+---------------------+



#### **Question 4**: What is the length of the longest trip in the dataset in hours?

```Ans : 162```

In [ ]:
df.join(zone_df, df.PULocationID == zone_df.LocationID, "left")\
  .groupby("Zone")\
  .agg(f.count(df.VendorID).alias("Number of Rides"))\
  .sort(f.asc("Number of Rides"))\
  .show(1)

+--------------------+---------------+
|                Zone|Number of Rides|
+--------------------+---------------+
|Governor's Island...|              1|
+--------------------+---------------+
only showing top 1 row



#### **Question 6**: What is the name of the LEAST frequent pickup location Zone?
```Ans: Governor's Island/Ellis Island/Liberty Island```